# imports/setup

In [20]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math
from datamodel import Listing, ConversionObservation
from main_tutorial_3 import Trader
from backtester import Backtester
import io

# backtester

# trader

In [ ]:
class Product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    STARFRUIT = "STARFRUIT"


PARAMS = {
    Product.RAINFOREST_RESIN: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0.5,
        "volume_limit": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,# joins orders within this edge 
        "default_edge": 4
    },
    Product.STARFRUIT: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        # for making
        "disregard_edge": 1,
        "join_edge": 3,
        "default_edge": 5,
    },
}

# backtest run

In [22]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

### setup

In [ ]:
def calculate_starfruit_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_RAINFOREST_RESIN_fair(order_depth):
    return 10000

In [ ]:
listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS'),
    'ORCHIDS': Listing(symbol='ORCHIDS', product='ORCHIDS', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 20,
    'STARFRUIT': 20
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_RAINFOREST_RESIN_fair,
    "STARFRUIT": calculate_starfruit_fair
}

In [25]:

market_data, trade_history = _process_data_('./tutorial.log')
trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, './backtestruns/round2_arb_bt')
backtester.run()
print(backtester.pnl)

KeyError: 'RAINFOREST_RESIN'

In [ ]:
# with fair prediction
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)

trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)


{'AMETHYSTS': 13943, 'STARFRUIT': 13361.0}


`{'AMETHYSTS': 14554, 'STARFRUIT': 14144.5}`

In [ ]:
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)

trader = Trader()
backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)

{'AMETHYSTS': 14554.0, 'STARFRUIT': 14118.0}


# backtest gridsearch

In [ ]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [ ]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [ ]:
listings = {
    'AMETHYSTS': Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS')
}

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20
}

fair_calculations = {
    "AMETHYSTS": calculate_amethysts_fair,
    "STARFRUIT": calculate_starfruit_fair
}


In [ ]:
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)



### run

In [ ]:
backtest_dir = "backtestruns"

param_grid = {
    Product.AMETHYSTS: {
        "fair_value": [10000],
        "take_width": [1],
        "clear_width": [0.5],
        "volume_limit": [0],
        # for making
        "disregard_edge": [1],  # disregards orders for joining or pennying within this value from fair
        "join_edge": [2],# joins orders within this edge 
        "default_edge": [4]
    },
    Product.STARFRUIT: {
        "take_width": [1],
        "clear_width": [0, -0.25],
        "prevent_adverse": [True],
        "adverse_volume": [15],
        "reversion_beta": [-0.229],
        # for making
        "disregard_edge": [1],
        "join_edge": [3],
        "default_edge": [5],
    },
}



trader = Trader()

amethyst_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "AMETHYSTS")
print("AMETHYSTS results:")
for params, pnl in amethyst_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

starfruit_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "STARFRUIT")
print("STARFRUIT results:")
for params, pnl in starfruit_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

Running backtests for AMETHYSTS: 100%|██████████| 1/1 [00:12<00:00, 12.17s/backtest]


AMETHYSTS results:
{'fair_value': 10000, 'take_width': 1, 'clear_width': 0.5, 'volume_limit': 0, 'disregard_edge': 1, 'join_edge': 2, 'default_edge': 4}
pnl: 13943


Running backtests for STARFRUIT: 100%|██████████| 2/2 [00:21<00:00, 11.00s/backtest]

STARFRUIT results:
{'take_width': 1, 'clear_width': 0, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5}
pnl: 13361.0
{'take_width': 1, 'clear_width': -0.25, 'prevent_adverse': True, 'adverse_volume': 15, 'reversion_beta': -0.229, 'disregard_edge': 1, 'join_edge': 3, 'default_edge': 5}
pnl: 13503.0


## analyze

In [ ]:

def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split('-', 1)
        params = dict(param.split('=') for param in params_str.split('-'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

AMETHYSTS_fair_value=10000_take_width=3_clear_width=0_volume_limit=15


ValueError: dictionary update sequence element #0 has length 1; 2 is required